In [1]:
foodname = "jabchae"

In [2]:
import os

photo_list = []

for f in os.listdir('./datasets/{}/'.format(foodname)):
    if 'png' in f:
        photo_list.append(f)

In [3]:
photo_list

['jabchae.0.png',
 'jabchae.1.png',
 'jabchae.10.png',
 'jabchae.100.png',
 'jabchae.1000.png',
 'jabchae.1001.png',
 'jabchae.1002.png',
 'jabchae.1003.png',
 'jabchae.1004.png',
 'jabchae.1005.png',
 'jabchae.1006.png',
 'jabchae.1007.png',
 'jabchae.1008.png',
 'jabchae.1009.png',
 'jabchae.101.png',
 'jabchae.1010.png',
 'jabchae.1011.png',
 'jabchae.1012.png',
 'jabchae.1013.png',
 'jabchae.1014.png',
 'jabchae.1015.png',
 'jabchae.1016.png',
 'jabchae.1017.png',
 'jabchae.1018.png',
 'jabchae.1019.png',
 'jabchae.102.png',
 'jabchae.1020.png',
 'jabchae.1021.png',
 'jabchae.1022.png',
 'jabchae.1023.png',
 'jabchae.1024.png',
 'jabchae.1025.png',
 'jabchae.1026.png',
 'jabchae.1027.png',
 'jabchae.1028.png',
 'jabchae.1029.png',
 'jabchae.103.png',
 'jabchae.1030.png',
 'jabchae.1031.png',
 'jabchae.1032.png',
 'jabchae.1033.png',
 'jabchae.1034.png',
 'jabchae.1035.png',
 'jabchae.1036.png',
 'jabchae.1037.png',
 'jabchae.1038.png',
 'jabchae.1039.png',
 'jabchae.104.png',
 'jab

In [4]:
photo_size = list(map(lambda x: os.path.getsize('./datasets/{}'.format(foodname) + '/' + x), photo_list))

In [5]:
import pandas as pd

# Find Same Photos
fsp = pd.DataFrame({'filename_raw':photo_list, 'size':photo_size})

print('사진의 갯수 :', len(fsp))

사진의 갯수 : 5325


In [6]:
import re   # 정규표현식
com = re.compile(' \d')
fsp['filename'] = list(map(lambda x: com.sub('', x), photo_list))

In [7]:
# Photo Value Counts
pvc = pd.DataFrame({'filename':fsp['filename'].value_counts().index, 'fn_counts':fsp['filename'].value_counts().values})   
psvc = pd.DataFrame({'size':fsp['size'].value_counts().index, 'size_counts':fsp['size'].value_counts().values})   

fsp = pd.merge(fsp, pvc, how = 'left', on = 'filename')
fsp = pd.merge(fsp, psvc, how = 'left', on = 'size')

fsp.sample(2)

,filename_raw,size,filename,fn_counts,size_counts
1480,jabchae.233.png,24773,jabchae.233.png,1,1
1754,jabchae.2577.png,21435,jabchae.2577.png,1,1


In [8]:
for i in range(len(fsp)):
    if (fsp['fn_counts'][i] > 1) & (fsp['size_counts'][i] == 1):
        print(i)

In [9]:
# Find Same Phto_Not Same Name
fsp_nsn = fsp.sort_values(['filename_raw'], ascending = False).drop_duplicates(['filename'], keep = 'first')

In [10]:
print('남은 사진의 갯수 : {}\n지워진 사진의 갯수 : {}'.format(len(fsp_nsn), len(fsp)-len(fsp_nsn)))

남은 사진의 갯수 : 5325
지워진 사진의 갯수 : 0


In [11]:
pvc_nsn = pd.DataFrame({'filename':fsp_nsn['filename'].value_counts().index, 'fn_counts_nsn':fsp_nsn['filename'].value_counts().values})   
psvc_nsn = pd.DataFrame({'size':fsp_nsn['size'].value_counts().index, 'size_counts_nsn':fsp_nsn['size'].value_counts().values})   

fsp_nsn = pd.merge(fsp_nsn, pvc_nsn, how = 'left', on = 'filename')
fsp_nsn = pd.merge(fsp_nsn, psvc_nsn, how = 'left', on = 'size')

In [12]:
fsp_nsn[fsp_nsn['fn_counts_nsn']!=1]

,filename_raw,size,filename,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn


In [13]:
print('사이즈 겹치는 사진의 갯수 :', len(fsp_nsn[fsp_nsn['size_counts_nsn']!=1]))
print('중복 사이즈의 갯수 :', len(psvc_nsn[psvc_nsn['size_counts_nsn']>1]))

사이즈 겹치는 사진의 갯수 : 1525
중복 사이즈의 갯수 : 705


In [14]:
import cv2   # OpenCV
from skimage.measure import compare_ssim

# 삭제될 사진의 리스트
delete = []


for i in range(len(psvc_nsn)):
    
    # 중복된 크기(size)가 있는 경우
    if psvc_nsn['size_counts_nsn'][i] == 2:
        
        # 그 크기에 해당하는 사진을 불러온다. 
        temp = fsp_nsn[fsp_nsn['size']==psvc_nsn['size'][i]].reset_index(drop = True).sort_values(['filename'])
        
        # 사진 읽기
        imageA = cv2.imread('./datasets/{}/'.format(foodname)+temp['filename_raw'][0])
        imageB = cv2.imread('./datasets/{}/'.format(foodname)+temp['filename_raw'][1])
        
        # 이미지를 grayscale로 변환
        grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        
        # 이미지의 구조가 같다면 이미지 비교
        if len(grayA)==len(grayB):
            (score, diff) = compare_ssim(grayA, grayB, full=True)
            
            # 차이가 없다면 하나는 delete에 넣어주기
            if score == 1:
                delete.append(temp['filename_raw'][1])
            
            # 구조가 같지만 차이가 존재한다면 직접 확인하기     
            else:
                print('확인해보시오! : ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score : {score})')

C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

확인해보시오! :  jabchae.4968.png / jabchae.2112.png (score : 0.24715940400850772)


C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

확인해보시오! :  jabchae.591.png / jabchae.5643.png (score : 0.1690204667231752)


C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

...

C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

확인해보시오! :  jabchae.2064.png / jabchae.1783.png (score : 0.12964293653640038)


C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

확인해보시오! :  jabchae.3562.png / jabchae.2907.png (score : 0.07401206792658645)
확인해보시오! :  jabchae.640.png / jabchae.3153.png (score : 0.1140939749576372)


C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users\SMT021\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
C:\Users

In [15]:
try:
    os.mkdir("./datasets/{}/Result/".format(foodname))
except:
    print("이미 있음")
    
try:
    os.mkdir("./datasets/{}/Delete/".format(foodname))
except:
    print("이미 있음")

In [16]:
# 중복제거된 것들은 delete 리스트에 넣어주기
delete = delete + list(fsp[~fsp['filename_raw'].isin(fsp_nsn['filename_raw'])]['filename_raw'])

print('삭제할 목록 :', len(delete))

삭제할 목록 : 356


In [17]:
# result : 처음(fsp)데이터에서 - delete를 제외한 것
result = list(fsp[~fsp['filename_raw'].isin(delete)]['filename_raw'])

print('남길 목록 : ', len(result))

남길 목록 :  4969


In [18]:
import shutil

for i in result:
    shutil.move('./datasets/{}/'.format(foodname)+i, './datasets/{}/Result/'.format(foodname))
    
for i in delete:
    shutil.move('./datasets/{}/'.format(foodname)+i, './datasets/{}/Delete/'.format(foodname)) 